In [1]:
from math import log,sqrt,exp
from scipy import stats
from scipy.optimize import fsolve

In [5]:
class call_option(object):
    ''' Class for European call options in BSM model
    
    
    Attributes
    ====================
    s0: float
        initial stock/index level
        
    K: float
        strike price
    
    t: datetime/Timestamp object
        price date
    
    r: float
        constant risk-free short rate
    
    sigma: float
        volatility factor in diffusion term
        
        
    Methods
    =======================
    value: float
        returns present value of call option
        
    vega: float
        returns vega of call option
    
    imp_vol: float
        return implied volatility given option quote
    '''
    
    def __init__(self,s0,K,t,M,r,sigma):
        self.s0 = float(s0)
        self.K = K
        self.t = t
        self.M = M
        self.r = r
        self.sigma = sigma
    
    def update_ttm(self):
        ''' Updates time-to-maturity self.T'''
        if self.t > self.M:
            raise ValueError('Pricing date later than maturity')
        self.T = (self.M - self.t).days / 365
    
    def d1(self):
        ''' Helper function '''
        d1 = ((log(self.s0/self.K) 
            + (self.r + 0.5 * self.sigma ** 2) * self.T) 
            / (self.sigma * sqrt(self.T)))
    
        return d1
    
    def value(self):
        ''' return option value. '''
        self.update_ttm()
        d1 = self.d1()
        d2 = ((log(self.s0/self.K) 
            + (self.r - 0.5 * self.sigma ** 2) * self.T) 
             / (self.sigma * sqrt(self.T)))
        value = (self.s0 * stats.norm.cdf(d1,0.0,1.0)
                - self.K * exp(-self.r * self.T) * stats.norm.cdf(d2,0.0,1.0))
        return value
    
    def vega(self):
        ''' Return Vega of option.'''
        self.update_ttm()
        d1 = self.d1()
        vega = self.s0 * stats.norm.pdf(d1,0.0,1.0) * sqrt(self.T)
        return vega
        
        
    def imp_vol(self,c0,sigma_est=0.2):
        ''' Return implied volatility given option price. '''
        option = call_option(self.s0, self.K, self.t, self.M, 
                             self.r, sigma_est)
        option.update_ttm()
        def difference(sigma):
            option.sigma = sigma
            return option.value() - c0
        iv = fsolve(difference, sigma_est)[0]
        return iv

        
        